In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
np.seterr(divide='ignore')

data = pd.read_csv('C:/Users/david/Desktop/maturitni_prace/data_analysis_python/eff_october.csv', sep=',', header=None)

names = []
energy = np.array([])
eff = np.array([])
yerr = []
for i in range(3):
    names.append(data[i][0])
for j in range(1,31):
    eff = np.append(eff, float(data[0][j]))
    yerr = np.append(yerr, float(data[1][j]))
    energy = np.append(energy, float(data[2][j]))

In [3]:
def func(x, a, b, c, d, e, f):
    return a * np.log(x)+ b * pow(np.log(x), 2) + c * pow(np.log(x), 3) + d * pow(np.log(x), 4) + e *pow(np.log(x), 5) + f

output = []
pars, cov = curve_fit(f=func, xdata=energy, ydata=eff, sigma=1./(np.log((yerr*eff))**2))
coeff_err = np.sqrt(np.diag(cov))
'''
for i in range(len(pars)):
    output.append('a{0} = {1:.6f} +- {2:.7f} ({3:d} %)'.format(i, pars[i], coeff_err[i], abs(int(coeff_err[i]/pars[i]*100))))
for i in output:
    print(i)
'''

global eff_param
eff_param = pars

a0 = -4.122191 +- 0.7323317 (17 %)
a1 = 1.620808 +- 0.2766304 (17 %)
a2 = -0.312858 +- 0.0518798 (16 %)
a3 = 0.029710 +- 0.0048319 (16 %)
a4 = -0.001113 +- 0.0001788 (16 %)
a5 = 4.112870 +- 0.7698687 (18 %)


In [5]:
file1 = open(r'C:\Users\david\Desktop\maturitni_prace\UmassCoefff.txt', 'r')
#soubor skopirovaných dat z https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z92.html
energy_as_x_to_masscoeff = [] #MeV
masscoef_U = [] #(cm2/g)
v1_density = 18.37414276 #g/cm3
v1_height =  9.99 / 10 #cm

def linear(x, a, b):
    return a * (x) + b

for line in file1:
    energy_as_x_to_masscoeff.append(float(line[3:14]))
    masscoef_U.append(float(line[15:25]))

In [6]:
def calculate_absorbtion(energy, energy_x_values = energy_as_x_to_masscoeff, D = v1_height, density = v1_density):
    energy = energy/1000    #prevede keV na MeV
    for k in range(len(energy_x_values)):
        eng = energy_x_values[k]
        if eng > energy:
            pars, cov = curve_fit(f=linear, xdata=[last_eng, eng], ydata=[masscoef_U[k-1], masscoef_U[k]])
            masscoef_of_U = linear(energy, *pars)

            return (masscoef_of_U * density * D)/(1 - np.exp(-(masscoef_of_U * density * D)))
        
        last_eng = eng

print(calculate_absorbtion(185.712, D = 9.99 / 10))
print(calculate_absorbtion(766.38, D = 9.99 / 10))
print(calculate_absorbtion(185.712, D = 10.02 / 10, density= 18.60655066))
print(calculate_absorbtion(766.38, D = 10.02 / 10, density= 18.60655066))

30.608018026179913
2.321943623537837
31.088246380832555
2.3471096295283784


c:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
